# Dask Array

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

In [ ]:
%%time
arr_np = np.random.normal(0,1,size=(1_000,300_000))


CPU times: user 133 ms, sys: 9.95 ms, total: 142 ms
Wall time: 147 ms


In [ ]:
%%time
arr_da = da.random.normal(0,1,size=(1_000,300_000), chunks=(1_000,30_000))


CPU times: user 4.73 ms, sys: 0 ns, total: 4.73 ms
Wall time: 4.44 ms


In [ ]:
arr_da

dask.array<normal, shape=(1000, 300000), dtype=float64, chunksize=(1000, 30000), chunktype=numpy.ndarray>

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

In [ ]:
%%time
np.power(arr_np,2).sum()

CPU times: user 33.7 ms, sys: 5 µs, total: 33.7 ms
Wall time: 33.9 ms


2997571.4803146413

In [ ]:
%%time
da.power(arr_da,2).sum().compute()

CPU times: user 23.5 s, sys: 345 ms, total: 23.9 s
Wall time: 12.2 s


300024167.7728329

3. Визуализируйте граф вычислений для задачи 2.

dask.array<normal, shape=(1000, 300000), dtype=float64, chunksize=(1000, 30000), chunktype=numpy.ndarray>

## Лабораторная работа 11

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
import dask.array as da
import dask
import h5py
from functools import reduce
import numpy as np
from tqdm import tqdm # для отображения прогресса

In [ ]:
lst=range(10000000)
for i in tqdm(lst): # то, как это делается
  pass

100%|██████████| 100000000/100000000 [00:26<00:00, 3839050.41it/s]


1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [42]:
# filk= h5py.File("/content/drive/MyDrive/Colab Notebooks/lab 11/minutes_n_ingredients_full.hdf5", 'r')




def load ():
  with h5py.File("/content/drive/MyDrive/Colab Notebooks/lab 11/minutes_n_ingredients_full.hdf5", "r") as f:
    data = f['recipe']
    print(type(data))
  return data
#   #...
  

data=load()

# recipe_da = da(data,chunks=(100_000, 3)).compute()
# recipe_da

<class 'h5py._hl.dataset.Dataset'>


In [20]:
f = h5py.File("/content/drive/MyDrive/Colab Notebooks/lab 11/minutes_n_ingredients_full.hdf5", "r")
data = f['recipe']
data = da.from_array(data,chunks=(100_000, 3)).compute()
#....
#data = f['recipe']
print (type(data))
f.close()

<class 'numpy.ndarray'>


In [28]:
temp_file = h5py.File("/content/drive/MyDrive/Colab Notebooks/lab 11/minutes_n_ingredients_full.hdf5", "r" )
temp_dataset = h5py.File("recipes.hdf5", "w")
temp_dataset.create_dataset("recipes", None, None,temp_file['recipe'], chunks=True)
Chunks_Temp_files = h5py.File("recipes.hdf5", "r")


In [46]:
data

array([[ 683970,      33,       9],
       [1089012,      23,       5],
       [1428572,       0,       5],
       ...,
       [1910650,      60,       2],
       [ 713836,       0,       9],
       [ 660699,      64,       8]])

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [47]:
da.mean(data, axis=0)

dask.array<mean_agg-aggregate, shape=(3,), dtype=float64, chunksize=(3,), chunktype=numpy.ndarray>

In [21]:
res=da.mean(data, axis=0).compute()
a=res[1]
b=res[2]
print (res)

[1.12684089e+06 1.00420805e+03 5.41980080e+00]


3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [18]:
da.from_array(data)

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(2231637, 3), chunktype=numpy.ndarray>

In [16]:
da.from_array(data,chunks=(100_000, 3))

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

4. Выберите рецепты, время выполнения которых меньше медианного значения

In [ ]:
#ASK

re=data[:,1]
ress=da.from_array(re)
da.median(ress)
#print (da.median(data[:,1]))


In [50]:
ka=data[:,1]
ba=data[:,2]

med=np.median(ka)
ded=np.median(ba)

print (ka[ka < med])
print (ba[ba < ded])
result1=ka[ka < med]
result2=ba[ba < ded]

[23  0 24 ... 19  1  0]
[1 1 4 ... 4 1 2]



5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [113]:
# da.unique(data).compute()
# unique, counts = da.unique(data, return_counts=True)
# dict(zip(unique, counts))

#a = вф.array([0, 3, 0, 1, 0, 1, 2, 1, 0, 0, 0, 0, 1, 3, 4])
unique, counts = np.unique(data[:,1:2], return_counts=True)
print (da.asarray((unique, counts)).T)
ree=da.asarray((unique, counts), chunks=(1000,1000)).T.compute()
ree

dask.array<transpose, shape=(41, 2), dtype=int64, chunksize=(41, 2), chunktype=numpy.ndarray>


array([[     1, 222071],
       [     2, 224158],
       [     3, 229388],
       [     4, 234948],
       [     5, 240720],
       [     6, 244360],
       [     7, 247181],
       [     8, 246747],
       [     9, 246816],
       [    10,  22430],
       [    11,  19094],
       [    12,  15165],
       [    13,  11640],
       [    14,   8284],
       [    15,   6014],
       [    16,   4145],
       [    17,   2793],
       [    18,   1913],
       [    19,   1279],
       [    20,    852],
       [    21,    529],
       [    22,    346],
       [    23,    244],
       [    24,    178],
       [    25,    107],
       [    26,     68],
       [    27,     55],
       [    28,     33],
       [    29,     22],
       [    30,     20],
       [    31,     13],
       [    32,      5],
       [    33,      4],
       [    34,      3],
       [    35,      4],
       [    36,      1],
       [    37,      2],
       [    38,      1],
       [    39,      1],
       [    40,      2],


In [88]:
unique, counts = np.unique(data[:,1:2], return_counts=True)
np.asarray((unique, counts)).T


array([[         0,      31645],
       [         1,      31406],
       [         2,      32740],
       ...,
       [    288000,          1],
       [   1051200,          1],
       [2147483647,          1]])

In [98]:
unique = np.unique(data[:,2])
ree=da.asarray(unique).compute()
print (ree)
print (len(ree), " length")

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 43]
41  length


In [112]:
unique, counts = np.unique(data[:,2], return_counts=True)
print (da.asarray((unique, counts)))
ree=da.asarray((unique, counts)).T.compute()
ree

dask.array<array, shape=(2, 41), dtype=int64, chunksize=(2, 41), chunktype=numpy.ndarray>


array([[     1, 222071],
       [     2, 224158],
       [     3, 229388],
       [     4, 234948],
       [     5, 240720],
       [     6, 244360],
       [     7, 247181],
       [     8, 246747],
       [     9, 246816],
       [    10,  22430],
       [    11,  19094],
       [    12,  15165],
       [    13,  11640],
       [    14,   8284],
       [    15,   6014],
       [    16,   4145],
       [    17,   2793],
       [    18,   1913],
       [    19,   1279],
       [    20,    852],
       [    21,    529],
       [    22,    346],
       [    23,    244],
       [    24,    178],
       [    25,    107],
       [    26,     68],
       [    27,     55],
       [    28,     33],
       [    29,     22],
       [    30,     20],
       [    31,     13],
       [    32,      5],
       [    33,      4],
       [    34,      3],
       [    35,      4],
       [    36,      1],
       [    37,      2],
       [    38,      1],
       [    39,      1],
       [    40,      2],



6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [40]:
ogran=np.quantile(data[:,1], 0.75)
print ((data[data < ogran]))
re=data[data < ogran]
np.max(re)


[33  9 23 ...  0  9  8]


48

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

In [118]:
x = da.random.random((1, 2), chunks=(1, 1)).compute()
#print (x[0])
close_0=1000000000
close_1=1000000000
print (x[0][0] , x[0][1])
id=-100
for i in data:
  if (abs(x[0][0]-i[1])<close_0 and abs(x[0][1]-i[2])<close_1 ):
    close_0=abs(x[0][0]-i[1])
    close_1=abs(x[0][1]-i[2])
    id=i[0]
    print (i[1], i[2])

print (id , close_0, close_1)

0.5392221413569714 0.4507249317742412
33 9
23 5
11 4
5 3
3 1
2071147 2.4607778586430284 0.5492750682257588


In [124]:
x =[[200,43]]
close_0=1000000000
close_1=1000000000
id=0
for i in data:
    if (abs(x[0][0]-i[1])<close_0 and abs(x[0][1]-i[2])<close_1 ):
        close_0=abs(x[0][0]-i[1])
        close_1=abs(x[0][1]-i[2])
        print (i[1] , i[2])
        id=i[0]

print (id)

33 9
47 10
80 13
200 19
29046


In [ ]:

preferences = da.random.random((1, 2), chunks=(1, 1)).compute()
distance = data().to_dask_dataframe().iloc[:, 1:].sub(preferences)
print ( data().to_dask_dataframe().loc[ distance.apply(np.linalg.norm, axis=1, meta='int').idxmin().compute()].compute() )


8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом;
2. агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных.

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент

In [ ]:
Lenn = ((Chunks_Temp_files['recipes'][part][:, 1].sum(), Chunks_Temp_files['recipes'][part][:, 1].size) for part in Chunks_Temp_files['recipes'].iter_chunks())
sum_for_minutes = reduce (lambda mas, l: (mas[0] + l[0], mas[1] + l[1]), Lenn)
print ( sum_for_minutes[0] / sum_for_minutes[1])  